In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# validation dataset mse is improved for predicting 6 hour data more than other future dataset by using previous 10 datapoint #

# Remaining work -  Required ensemble techniques for hypertunning on validation dataset before checking for test dataset#

In [ ]:
table_result = pd.DataFrame()
table_result['prediction'] = ['6 hour','1 day', '3 day','5 day','6 hour','1 day', '3 day','5 day']
table_result['model'] = ['Randomforest','Randomforest','Randomforest','Randomforest','Linearregression','Linearregression','Linearregression','Linearregression']
table_result['without previous data (val mse)'] = ['0.0233','0.0224',"0.0312",'0.0331','0.0252','0.0208','0.0297','0.0322']
table_result['with previous data (val mse)'] = ['0.0116','0.0205','0.0205','0.0250','0.0151','0.0202','0.0267','0.0240']
table_result

,prediction,model,without previous data (val mse),with previous data (val mse)
0,6 hour,Randomforest,0.0233,0.0116
1,1 day,Randomforest,0.0224,0.0205
2,3 day,Randomforest,0.0312,0.0205
3,5 day,Randomforest,0.0331,0.0250
4,6 hour,Linearregression,0.0252,0.0151
5,1 day,Linearregression,0.0208,0.0202
6,3 day,Linearregression,0.0297,0.0267
7,5 day,Linearregression,0.0322,0.0240


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.linear_model import LinearRegression,Ridge
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score,precision_recall_curve,mean_absolute_error

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
# for 6 hour

df = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/train_preprocessed.csv')

df["date_time"]=pd.to_datetime(df.date_time)
df.set_index("date_time",inplace=True)

df['fog_index_future']= df['fog_index'].shift(-1)

# for without previous value just comment below 10 lines
df['fog_index_1'] =  df['fog_index'].shift(1)
df['fog_index_2'] =  df['fog_index'].shift(2)
df['fog_index_3'] =  df['fog_index'].shift(3)
df['fog_index_4'] =  df['fog_index'].shift(4)
df['fog_index_5'] =  df['fog_index'].shift(5)
df['fog_index_6'] =  df['fog_index'].shift(6)
df['fog_index_7'] =  df['fog_index'].shift(7)
df['fog_index_8'] =  df['fog_index'].shift(8)
df['fog_index_9'] =  df['fog_index'].shift(9)
df['fog_index_10'] =  df['fog_index'].shift(10)


df.dropna(inplace=True)

# selecting winter month(NDJF)
train =  df[(df.index.month==1) | (df.index.month==2)|(df.index.month==11) |(df.index.month==12)]


X_train,y_train = train.drop(["fog_index_future"],axis=1),train["fog_index_future"]

# taking validation dataset
val = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/val_preprocessed.csv')

val["date_time"]=pd.to_datetime(val.date_time)
val.set_index("date_time",inplace=True)

val['fog_index_future']= val['fog_index'].shift(-1)

# for without previous value just comment below 10 lines
val['fog_index_1'] =  val['fog_index'].shift(1)
val['fog_index_2'] =  val['fog_index'].shift(2)
val['fog_index_3'] =  val['fog_index'].shift(3)
val['fog_index_4'] =  val['fog_index'].shift(4)
val['fog_index_5'] =  val['fog_index'].shift(5)
val['fog_index_6'] =  val['fog_index'].shift(6)
val['fog_index_7'] =  val['fog_index'].shift(7)
val['fog_index_8'] =  val['fog_index'].shift(8)
val['fog_index_9'] =  val['fog_index'].shift(9)
val['fog_index_10'] =  val['fog_index'].shift(10)


val.dropna(inplace=True)

# selecting winter month(NDJF)
validate =  val[(val.index.month==1) | (val.index.month==2)|(val.index.month==11) |(val.index.month==12)]


X_val,y_val = validate.drop(["fog_index_future"],axis=1),validate["fog_index_future"]


rf = RandomForestRegressor()


rf.fit(X_train,y_train)

print('RandomforestRegressor')
print(metrics.mean_squared_error(y_train,rf.predict(X_train)))
print(metrics.mean_squared_error(y_val,rf.predict(X_val)))

lr = LinearRegression()


lr.fit(X_train,y_train)

print('LinearRegressor')
print(metrics.mean_squared_error(y_train,lr.predict(X_train)))
print(metrics.mean_squared_error(y_val,lr.predict(X_val)))

LinearRegressor
0.009245414377130789
0.01514001794335759


In [ ]:
# RandomForest
# 0.001674744328589142
# 0.023374047603179862
# after using previous value
# RandomForest
# 0.0009875065094650836
# 0.011618865062274771
# LinearRegressor
# 0.01310605891981658
# 0.025232608601949577
# after using previous value
# LinearRegressor
# 0.009245414377130789
# 0.01514001794335759

In [ ]:
validate['pred']=  rf.predict(X_val)

In [ ]:
# with previous data 6 hour prediction
import plotly.express as px
fig = px.line(validate.reset_index(),x='date_time',y=['fog_index_future','pred'])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1,label='1y',step='year',stepmode='todate'),
            dict(count=2,label='2y',step='year',stepmode='todate'),
            dict(count=3,label='3y',step='year',stepmode='todate'),
            dict(step='all')
        ])
    )
)
fig.show()

In [ ]:
# for 1 day

df = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/train_preprocessed.csv')

df["date_time"]=pd.to_datetime(df.date_time)
df.set_index("date_time",inplace=True)

df['fog_index_future']= df['fog_index'].shift(-4)

# for without previous value just comment below 10 lines
df['fog_index_1'] =  df['fog_index'].shift(4)
df['fog_index_2'] =  df['fog_index'].shift(8)
df['fog_index_3'] =  df['fog_index'].shift(12)
df['fog_index_4'] =  df['fog_index'].shift(16)
df['fog_index_5'] =  df['fog_index'].shift(20)
df['fog_index_6'] =  df['fog_index'].shift(24)
df['fog_index_7'] =  df['fog_index'].shift(28)
df['fog_index_8'] =  df['fog_index'].shift(32)
df['fog_index_9'] =  df['fog_index'].shift(36)
df['fog_index_10'] =  df['fog_index'].shift(40)


df.dropna(inplace=True)

# selecting winter month(NDJF)
train =  df[(df.index.month==1) | (df.index.month==2)|(df.index.month==11) |(df.index.month==12)]


X_train,y_train = train.drop(["fog_index_future"],axis=1),train["fog_index_future"]

# taking validation dataset
val = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/val_preprocessed.csv')

val["date_time"]=pd.to_datetime(val.date_time)
val.set_index("date_time",inplace=True)

val['fog_index_future']= val['fog_index'].shift(-4)

# for without previous value just comment below 10 lines
val['fog_index_1'] =  val['fog_index'].shift(4)
val['fog_index_2'] =  val['fog_index'].shift(8)
val['fog_index_3'] =  val['fog_index'].shift(12)
val['fog_index_4'] =  val['fog_index'].shift(16)
val['fog_index_5'] =  val['fog_index'].shift(20)
val['fog_index_6'] =  val['fog_index'].shift(24)
val['fog_index_7'] =  val['fog_index'].shift(28)
val['fog_index_8'] =  val['fog_index'].shift(32)
val['fog_index_9'] =  val['fog_index'].shift(36)
val['fog_index_10'] =  val['fog_index'].shift(40)


val.dropna(inplace=True)

# selecting winter month(NDJF)
validate =  val[(val.index.month==1) | (val.index.month==2)|(val.index.month==11) |(val.index.month==12)]


X_val,y_val = validate.drop(["fog_index_future"],axis=1),validate["fog_index_future"]


rf = RandomForestRegressor()


rf.fit(X_train,y_train)

print('RandomforestRegressor')
print(metrics.mean_squared_error(y_train,rf.predict(X_train)))
print(metrics.mean_squared_error(y_val,rf.predict(X_val)))

lr = LinearRegression()


lr.fit(X_train,y_train)

print('LinearRegressor')
print(metrics.mean_squared_error(y_train,lr.predict(X_train)))
print(metrics.mean_squared_error(y_val,lr.predict(X_val)))

RandomforestRegressor
0.0017430837963521711
0.022374467192282622
LinearRegressor
0.01201061659899603
0.020818037597615233


In [ ]:
# RandomforestRegressor
# 0.0017808884802742607
# 0.022435122805452686
# LinearRegressor
# 0.01201061659899603
# 0.020818037597615233
# after taking the previous 10 values
# RandomforestRegressor
# 0.0016569274813284947
# 0.020509109680352536
# LinearRegressor
# 0.011504554465845094
# 0.020286424797676412

In [ ]:
validate['pred']=  rf.predict(X_val)

In [ ]:
# with previous data 1 day prediction
import plotly.express as px
fig = px.line(validate.reset_index(),x='date_time',y=['fog_index_future','pred'])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1,label='1y',step='year',stepmode='todate'),
            dict(count=2,label='2y',step='year',stepmode='todate'),
            dict(count=3,label='3y',step='year',stepmode='todate'),
            dict(step='all')
        ])
    )
)
fig.show()

In [ ]:
# for 3 day

df = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/train_preprocessed.csv')

df["date_time"]=pd.to_datetime(df.date_time)
df.set_index("date_time",inplace=True)

df['fog_index_future']= df['fog_index'].shift(-12)

# for without previous value just comment below 10 lines
df['fog_index_1'] =  df['fog_index'].shift(12)
df['fog_index_2'] =  df['fog_index'].shift(24)
df['fog_index_3'] =  df['fog_index'].shift(36)
df['fog_index_4'] =  df['fog_index'].shift(48)
df['fog_index_5'] =  df['fog_index'].shift(60)
df['fog_index_6'] =  df['fog_index'].shift(72)
df['fog_index_7'] =  df['fog_index'].shift(84)
df['fog_index_8'] =  df['fog_index'].shift(96)
df['fog_index_9'] =  df['fog_index'].shift(108)
df['fog_index_10'] =  df['fog_index'].shift(120)


df.dropna(inplace=True)

# selecting winter month(NDJF)
train =  df[(df.index.month==1) | (df.index.month==2)|(df.index.month==11) |(df.index.month==12)]


X_train,y_train = train.drop(["fog_index_future"],axis=1),train["fog_index_future"]

# taking validation dataset
val = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/val_preprocessed.csv')

val["date_time"]=pd.to_datetime(val.date_time)
val.set_index("date_time",inplace=True)

val['fog_index_future']= val['fog_index'].shift(-12)

# for without previous value just comment below 10 lines
val['fog_index_1'] =  val['fog_index'].shift(12)
val['fog_index_2'] =  val['fog_index'].shift(24)
val['fog_index_3'] =  val['fog_index'].shift(36)
val['fog_index_4'] =  val['fog_index'].shift(48)
val['fog_index_5'] =  val['fog_index'].shift(60)
val['fog_index_6'] =  val['fog_index'].shift(72)
val['fog_index_7'] =  val['fog_index'].shift(84)
val['fog_index_8'] =  val['fog_index'].shift(96)
val['fog_index_9'] =  val['fog_index'].shift(108)
val['fog_index_10'] =  val['fog_index'].shift(120)


val.dropna(inplace=True)

# selecting winter month(NDJF)
validate =  val[(val.index.month==1) | (val.index.month==2)|(val.index.month==11) |(val.index.month==12)]


X_val,y_val = validate.drop(["fog_index_future"],axis=1),validate["fog_index_future"]


rf = RandomForestRegressor()


rf.fit(X_train,y_train)

print('RandomforestRegressor')
print(metrics.mean_squared_error(y_train,rf.predict(X_train)))
print(metrics.mean_squared_error(y_val,rf.predict(X_val)))

lr = LinearRegression()


lr.fit(X_train,y_train)

print('LinearRegressor')
print(metrics.mean_squared_error(y_train,lr.predict(X_train)))
print(metrics.mean_squared_error(y_val,lr.predict(X_val)))

RandomforestRegressor
0.0020668809354166717
0.027590433309369927
LinearRegressor
0.014761756676250118
0.02678505962636464


In [ ]:
# RandomforestRegressor
# 0.0022044609499461683
# 0.03127796430653653
# LinearRegressor
# 0.015350738256947717
# 0.029761850447663198
# after using previous value
# RandomforestRegressor
# 0.0020668809354166717
# 0.027590433309369927
# LinearRegressor
# 0.014761756676250118
# 0.02678505962636464

In [ ]:
validate['pred']=  rf.predict(X_val)

In [ ]:
# with previous data 3 day prediction
import plotly.express as px
fig = px.line(validate.reset_index(),x='date_time',y=['fog_index_future','pred'])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1,label='1y',step='year',stepmode='todate'),
            dict(count=2,label='2y',step='year',stepmode='todate'),
            dict(count=3,label='3y',step='year',stepmode='todate'),
            dict(step='all')
        ])
    )
)
fig.show()

In [ ]:
# for 3 day

df = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/train_preprocessed.csv')

df["date_time"]=pd.to_datetime(df.date_time)
df.set_index("date_time",inplace=True)

df['fog_index_future']= df['fog_index'].shift(-20)

# for without previous value just comment below 10 lines
df['fog_index_1'] =  df['fog_index'].shift(20)
df['fog_index_2'] =  df['fog_index'].shift(40)
df['fog_index_3'] =  df['fog_index'].shift(60)
df['fog_index_4'] =  df['fog_index'].shift(80)
df['fog_index_5'] =  df['fog_index'].shift(100)
df['fog_index_6'] =  df['fog_index'].shift(120)
df['fog_index_7'] =  df['fog_index'].shift(140)
df['fog_index_8'] =  df['fog_index'].shift(160)
df['fog_index_9'] =  df['fog_index'].shift(180)
df['fog_index_10'] =  df['fog_index'].shift(200)


df.dropna(inplace=True)

# selecting winter month(NDJF)
train =  df[(df.index.month==1) | (df.index.month==2)|(df.index.month==11) |(df.index.month==12)]


X_train,y_train = train.drop(["fog_index_future"],axis=1),train["fog_index_future"]

# taking validation dataset
val = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/val_preprocessed.csv')

val["date_time"]=pd.to_datetime(val.date_time)
val.set_index("date_time",inplace=True)

val['fog_index_future']= val['fog_index'].shift(-20)

# for without previous value just comment below 10 lines
val['fog_index_1'] =  val['fog_index'].shift(20)
val['fog_index_2'] =  val['fog_index'].shift(40)
val['fog_index_3'] =  val['fog_index'].shift(60)
val['fog_index_4'] =  val['fog_index'].shift(80)
val['fog_index_5'] =  val['fog_index'].shift(100)
val['fog_index_6'] =  val['fog_index'].shift(120)
val['fog_index_7'] =  val['fog_index'].shift(140)
val['fog_index_8'] =  val['fog_index'].shift(160)
val['fog_index_9'] =  val['fog_index'].shift(180)
val['fog_index_10'] =  val['fog_index'].shift(200)


val.dropna(inplace=True)

# selecting winter month(NDJF)
validate =  val[(val.index.month==1) | (val.index.month==2)|(val.index.month==11) |(val.index.month==12)]


X_val,y_val = validate.drop(["fog_index_future"],axis=1),validate["fog_index_future"]


rf = RandomForestRegressor()


rf.fit(X_train,y_train)

print('RandomforestRegressor')
print(metrics.mean_squared_error(y_train,rf.predict(X_train)))
print(metrics.mean_squared_error(y_val,rf.predict(X_val)))

lr = LinearRegression()


lr.fit(X_train,y_train)

print('LinearRegressor')
print(metrics.mean_squared_error(y_train,lr.predict(X_train)))
print(metrics.mean_squared_error(y_val,lr.predict(X_val)))

RandomforestRegressor
0.0022343885223757003
0.025031427095682287
LinearRegressor
0.015950614146154188
0.024063493327200727


In [ ]:
# RandomforestRegressor
# 0.002308066868644357
# 0.03311668938645286
# LinearRegressor
# 0.016023033097929244
# 0.032281505288929126
# after using previous value
# RandomforestRegressor
# 0.0022343885223757003
# 0.025031427095682287
# LinearRegressor
# 0.015950614146154188
# 0.024063493327200727

In [ ]:
validate['pred']=  rf.predict(X_val)

In [ ]:
# with previous data 5 day prediction
import plotly.express as px
fig = px.line(validate.reset_index(),x='date_time',y=['fog_index_future','pred'])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1,label='1y',step='year',stepmode='todate'),
            dict(count=2,label='2y',step='year',stepmode='todate'),
            dict(count=3,label='3y',step='year',stepmode='todate'),
            dict(step='all')
        ])
    )
)
fig.show()